In [1]:
%load_ext autoreload
%autoreload 2

from kasearch import AlignSequences, SearchDB, PrepareDB

## **Prepare query sequence (sequence to search with)**

In [2]:
raw_queries = [
    'QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS',
    'QVQLKESGPGLVAPSQSLSITCTVSGFSLTSYGVSWVRQPPGKGLEWLGVIWGDGSTNYHSALISRLSISKENSKSQVFLKLNSLQTDDTATYYCAKPGGDYWGQGTSVTVSS',
]

In [3]:
query_db = AlignSequences(raw_queries, # Sequences as strings to align.
                          n_jobs=1     # Allocated number for jobs/threads for the search.
                         )
query_db.db.aligned_seqs

array([[81, 86, 75,  0, 76, 81, 69, 83, 71, 65,  0, 69, 76, 65, 82, 80,
        71, 65, 83, 86, 75, 76, 83, 67, 75, 65, 83, 71, 89, 84, 70,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 84, 78, 89, 87, 77, 81,  0, 87,
        86, 75, 81,  0, 82,  0, 80,  0, 71,  0, 81,  0,  0, 71,  0, 76,
        68,  0, 87, 73, 71, 65, 73, 89, 80, 71,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, 68, 71, 78, 84, 82, 89,  0,  0, 84,  0,  0,
        72,  0,  0, 75, 70,  0,  0, 75,  0,  0,  0, 71, 75, 65, 84, 76,
        84, 65,  0, 68,  0,  0,  0, 75,  0, 83,  0,  0, 83, 83,  0,  0,
         0,  0, 84,  0, 65, 89, 77, 81, 76, 83, 83, 76, 65, 83,  0, 69,
        68, 83, 71, 86, 89, 89, 67, 65, 82, 71, 69, 71, 78,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, 89, 65, 87, 70, 65, 89, 87, 71,  0, 81,
        71, 84, 84, 86, 84, 86, 83, 83]], dtype=int8)

--------------
## **Initiate search class**

### Database to search against
- If no database path is give, a small OAS version will be downloaded to search against.
- The full version of OAS can be downloaded here ().
- You can also give it the path for a custom database to search against. (See below for how to create a custom database).

### Regions to search with
- Default regions are the whole chain, CDRs or CDR3.
- User-defined regions can be added, as seen with the paratope search below.
- For each region, the search can either be based on exact length match or not.
- For a more specific search, the search can be focused on a specific chain and species.

In [4]:
paratope = ["107 ", "108 ","111C", "114 ","115 "]

In [5]:
oasdb = SearchDB(
    #database_path='oasdb-small', # Path to your database. Default will be to download a small prepared version of OAS.
    allowed_chain='Heavy',         # Search against a specific chain. Default is any chain.
    allowed_species='Human',       # Search against a specific species. Default is any species.
    regions=['whole', 'cdrs', 'cdr3', paratope],
    length_matched=[False, True, True, True],
)

-----------
## **Run search**

A search takes ~23min per sequence against all of OAS and ~2min per sequence against the small OAS.

To specify the number of highest similar sequences to keep, you can change the keep_best_n parameter.

In [6]:
%%time 
oasdb.search(query_db.db.aligned_seqs, # Input can be a single or multiple aligned sequences at a time.
             keep_best_n=10,               # You can define how many most similar sequences to return
            )

CPU times: user 2min 34s, sys: 15.3 s, total: 2min 50s
Wall time: 1min 2s


### Get N best identities

Identities of the most similar sequence for each of the regions can be fetched from the object with the bellow command.

In [7]:
oasdb.current_best_identities

array([[[0.8333334, 0.8333333, 1.       ],
        [0.8333334, 0.8333333, 1.       ],
        [0.8333334, 0.8333333, 1.       ],
        [0.8333334, 0.8333333, 1.       ],
        [0.8333334, 0.8333333, 1.       ],
        [0.8333334, 0.75     , 1.       ],
        [0.8333333, 0.75     , 1.       ],
        [0.8333333, 0.75     , 1.       ],
        [0.8333333, 0.75     , 1.       ],
        [0.8333333, 0.75     , 1.       ]]], dtype=float32)

### Get ID's of sequences with the N best identities

Similarly, the ID's of the most similar sequences can be fetched from the object with the bellow command.

In [8]:
oasdb.current_best_ids

array([[[[   3574,     439],
         [   3580,    1736],
         [   8999,   34163]],

        [[   3574,   14330],
         [   3611,    2027],
         [  13687,   11125]],

        [[   3574,    4608],
         [   3574,   20140],
         [   4008,    1239]],

        [[   4059,    3259],
         [   3595,     904],
         [   2791,   20061]],

        [[   4059,   33660],
         [   3611,     391],
         [   5484,   88906]],

        [[   4059,     925],
         [  10012,   61994],
         [  11076,   69071]],

        [[   3580,    1736],
         [   3358, 1405904],
         [  11447,   69843]],

        [[   3611,    2027],
         [  10012, 1383316],
         [   4121,   10306]],

        [[   3611,     391],
         [   8614,   74371],
         [   9516, 1187568]],

        [[   3574,   20140],
         [   3358,  178333],
         [   3119, 3727905]]]], dtype=int32)

---------
## Extract the meta data from matched sequences

Using the get_meta function, the meta data for all matched sequences for each query and region can be extracted as seen below.

Zero (0) is the first query or the first region in the list when initiating the search class. 

NB: The column "sequence_alignment_aa" holds the antibody sequence.

In [9]:
n_best_sequences = oasdb.get_meta(n_query = 0,          # Which query to extract meta data from
                                  n_region = 0,         # Which region to extract meta data from
                                  n_sequences = 2,  # Number of sequences to extract (default is all, which is keep_best_n)
                                  n_jobs=10             # Allocated number for jobs/threads for the extraction
                                 )
n_best_sequences

,sequence,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call,d_call,...,Longitudinal,Age,Disease,Subject,Vaccine,Chain,Unique sequences,Total sequences,Isotype,Identity
0,GATGAGATCACTGTTCTCTTTACAGTTACTGAGCACACAGGACCTC...,H,F,T,F,T,F,F,IGHV1-2*02,IGHD3-3*01,...,no,no,POEMS,Patient_19,None,Heavy,38304,108623,Bulk,0.833333
1,GACAGTTACTGAGCACACAGGACCTCACCATGGGATGGAGCTGTAT...,H,F,T,F,T,F,F,IGHV1-2*02,IGHD3-3*01,...,no,no,POEMS,Patient_19,None,Heavy,38304,108623,Bulk,0.833333


In [10]:
n_best_sequences.sequence_alignment_aa.values

array(['QVQLQQPGAEVVKPGASVKLSCKASGYTFTSYWMHWVKQRPGRGLEWIGRIDPNSGGTKYNEKFKSKATLIVDKSSSTVYMQFSSLTSEDSAVYYCARGEWFAYWGQGTLVTVS',
       'QVQLQQPGAEVVKPGASVKLSCKASGYTFTSYWMHWVKQRPGRGLEWIGRIDPNSGGTKYNEKFKSKATLIVDKSSSTVYMKFSSLTSEDSAVYYCARGEWFAYWGQGTLVTVS'],
      dtype=object)

----------
## Create custom database


To create your own database you first need to create a csv file in the OAS format. For an example file, look at data/custom-data-example.csv. This file consists of a dictionary containing the metadata in the first line and then rows of the individual sequences afterwards. Only the Species and Chain is strictly needed in the metadata, and only the amino acids sequence of the antibodies is required for each antibody sequence.

### 1. Format your data into OAS files

In [11]:
import json
import os
import pandas as pd

from kasearch.species_anarci import number
from kasearch.merge_db import merge_files

In [12]:
metadata = {"Species":"Human", "Chain":"Heavy"}
metadata = pd.Series(name=json.dumps(metadata), dtype='object')
seqsdata = pd.DataFrame([["EVQLVESGGGLAKPGGSLRLHCAASGFAFSSYWMNWVRQAPGKRLEWVSAINLGGGLTYYAASVKGRFTISRDNSKNTLSLQMNSLRAEDTAVYYCATDYCSSTYCSPVGDYWGQGVLVTVSS"],
                          ["EVQLVQSGAEVKRPGESLKISCKTSGYSFTSYWISWVRQMPGKGLEWMGAIDPSDSDTRYNPSFQGQVTISADKSISTAYLQWSRLKASDTATYYCAIKKYCTGSGCRRWYFDLWGPGT"]
                         ], columns = ['sequence_alignment_aa'])

In [13]:
save_file = "../data/custom-data-examples.csv"
metadata.to_csv(save_file, index=False)
seqsdata.to_csv(save_file, index=False, mode='a')

### 2. Turn your OAS formatted files into a custom database

After creating all the files you want to include in the new database, you can run the following code to create the database.

In [14]:
db_folder = "../data/my_db"
db_files = ['../data/custom-data-examples.csv']

In [15]:
newDB = PrepareDB(db_path=db_folder)

db_dict = {}
for num, data_unit_file in enumerate(db_files):

    db_dict[num] = data_unit_file
    
    metadata = json.loads(','.join(pd.read_csv(data_unit_file, nrows=0).columns))
    seqsdata = pd.read_csv(data_unit_file, header=1, usecols=['sequence_alignment_aa']).iloc[:,0].values
    numbered_sequences = [number(sequence, allowed_species=None)[0] for sequence in seqsdata]

    newDB.prepare_database(numbered_sequences, 
                           file_id=num, 
                           chain=metadata['Chain'], 
                           species=metadata['Species'])
    
newDB.save_database()

merge_files(db_folder)

with open(os.path.join(db_folder, "id_to_study.txt"), "w") as handle: handle.write(str(db_dict))

### 3. Initiate the search class with your custom database


In [16]:
mydb = SearchDB(database_path=db_folder,      # Path to your database. Default will be to download a prepared version of OAS.
                 allowed_chain='Heavy',       # Search against a specific chain. Default is any chain.
                 allowed_species='Any',       # Search against a specific species. Default is any species.
                )